# Group project 1 - Conway's Game of Life
## Due January 24th, 10:30 AM

Conway's Game of Life is a "no-player never-ending game" played on board of squares.  It was invented by [John Conway](https://www.theguardian.com/science/2015/jul/23/john-horton-conway-the-most-charismatic-mathematician-in-the-world) in 1968, out of a desire to create a demonstration of how simple rules might lead to complex behavior, much like how relatively simple physical processes generated all of earth's complicated biodiversity, at least in microcosmic sense.  For our purposes, it will yield a fun and interesting starting point for practicing some useful python skills prior to diving into some more mathematical topics.  

The rules of the game are simple, and we'll develop them sequentially.  We begin by defining a rectangular board of cells, each of which can be either "alive" or "dead".  A good size to begin with is 10x10.  The best way to store this board is as a numpy array.  **Import numpy and create the board, with all positions initially set to "dead" (or false, as the case may be).**

In [11]:
import numpy as np



# Create a boolean numpy array to act as the board here, and initialize it to all false values.  You
# may find the commands np.zeros and the array's astype method to be useful here

board = np.full((10,10), False)

Now that we have the board, the next thing that we'll need to do is to define our initial configuration, or *state*.  While we will change this in the future, set a line of three adjacent cells either vertically or horizontally in the middle of the board.  You can use numpy's array indexing for this, e.g. if you wanted to initialize a 2x2 square with corner at position i=2,j=2, you could set S[2:4,2:4]=True, if S is the name of your board.

In [12]:
board[2:5,2] = True # keep in mind [2:5,#] means rows 2 through 4
board

array([[False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False,  True, False, False, False, False, False, False,
        False],
       [False, False,  True, False, False, False, False, False, False,
        False],
       [False, False,  True, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False]])

Now we get to the rules of the game.  Life is a dynamical system, which is to say that 
$$
S_{t+1} = F(S_t)
$$
or the next board state is a function of the current board state, where $S_t$ is the board state at an integer "time" $t$.  The rules that define $F$ are simple, and here we'll just state them in words rather than in equations.
 - If a cell is currently alive, and it has two or three neighbors (out of eight possible) that are also alive, then it remains alive.
 - If a cell is dead, but has three neighbors (out of eight possible) that are alive, then it becomes alive.
 - All other cells become or remain dead.  
 
It's also necessary to consider what happens at the edges of the domain.  You should use *periodic boundary conditions*, which is to say that if you are considering the neighbors of a cell that exists, for example on the right edge of the board, you would count its five neighboring cells on the rights side of the board, and also three cells on the left edge (and vice versa.  And analogously for the top and bottom).  **Create a python function that takes as an argument the existing board state and returns a new board state based on the rules described above.  Try to make this function as efficient as possible!** 

In [15]:
# import function from github, gets a list of neighbors from a 2D array
# https://gist.github.com/003random/3e97ddc449aa394ebf25edb25c8c6110
def neighbors(matrix, rowNumber, colNumber):
    result = []
    for rowAdd in range(-1, 2):
        newRow = rowNumber + rowAdd
        if newRow >= 0 and newRow <= len(matrix)-1:
            for colAdd in range(-1, 2):
                newCol = colNumber + colAdd
                if newCol >= 0 and newCol <= len(matrix)-1:
                    if newCol == colNumber and newRow == rowNumber:
                        continue
                    result.append(matrix[newCol][newRow])
    return result

[False, False, False]

In [18]:
def update_board(S):
    sz = len(S)
    s_new = np.full((sz,sz), False)
    S = np.vstack((S[sz-1,:],S,S[0,:]))
    S = np.vstack((S[:,sz-1,].reshape(sz+2,1), S, S[:,0].reshape(sz+2,1)))
    
    S_new = [0,0]
    
    # Check which cells satisfy rule 1
    # Check which cells satisfy rule 2
    # Decide which cells are alive and dead based on rules
    return S_new

update_board(board)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 10

Do a basic test of your function to ensure correctness.  The reason that I asked you to create a line of three pixels is because the rules above lead to a very simple transformation: a vertical line becomes a horizontal line, and a horizontal line becomes a vertical line.  It will be helpful to visualize this.  **Use matplotlib to visualize your board state both before and after application of the update_board function.** 

In [ ]:
import matplotlib.pyplot as plt

# plt.imshow is a particularly useful command for this

While this is interesting, the real fascinating dynamics of Conway's Game of Life emerge when the update_board function is iterated from time $t=0$ to $t=T$.  Write a for loop that performs this iteration and saves the output at each step to a list.  One common gotcha is that if you append your state array to a list, you may end up with $T$ references to the same array, which of course is being updated.  This is not what you want.  Be sure you save the board state at time $t$ by using a function such as np.copy prior to appending it.  Try running the code for 20 steps.  

It would be unwieldy to visualize each of the states of a time-evolving system individually.  A much more natural way of looking at them is to create an animation.  There are many ways to do this, but one nice way is to use the matplotlib FuncAnimation method.  All you need to do is to create an animate function, which takes as argument the frame number and updates a matplotlib plot.  Note that we need to put jupyter into matplotlib notebook mode to make this work (if we ran this code in a normal python interpreter, this wouldn't be required).  It's easier to show than to tell:

In [ ]:
import matplotlib.animation as anim
%matplotlib notebook

fig,ax = plt.subplots()
fig.set_size_inches(10,10)

im = ax.imshow(states[0])

def animate(frame_number):
    im.set_data(states[frame_number])
    return im
    
animation = anim.FuncAnimation(fig, animate, frames=len(states), interval=100)

Feel free to modify the parameters of the animate function, but if all has gone well, what you should be seeing is an oscilllating vertical and horizontal line of 3 pixels.  This isn't a very exciting result, but it should ensure that your code is running correctly. 

Now, let's do something more interesting.  Instead of a line, let's try as an initial state the R-pentomino, which would be
![pentomino](pentomino.png)
**Create a method that initializes the board state to one R-pentomino in the middle.  Rerun the above analysis and write a short qualitative characterization of the types of patterns that appear**.  If you wish, you can make the board larger, say size 100x100 and run the simulation for longer to get a better sense of the results.  

An easy way to get a sense of the diversity generated by Life's rules is to begin with a random board state.  **Generate a board of size 100x100 with each pixel either alive or dead with 50% probability**.  (A useful function for doing this might be numpy.random.randint).  **Run the game for 1000 iterations.  Write a characterization of the types of patterns that appear.  As a guide, there is a common vernacular for characterizing these different patterns: still life (stable and non-changing patterns), oscillators (you've already seen one example with the horizontal/vertical line), and gliders.  Describe some pixel patterns that fall into each of these categories.**

Next, let's ask an analytical question: Given the randomized board state above, does the total number of live pixels ever stabilize?  Stated another way, if I ran the simulation above 100 times, and queried the number of live pixels on the board at $t=1000$, would they be similar each time?  **Devise a way to address this question.  Write a short statement with your conclusion, and generate one figure using matplotlib that supports your analysis**.

Finally, one aspect of Life that I find interesting is the fragility of the rule set.  **Make some simple modification to the rules.  Write a short statement describing what you changed and how it affected the dynamics of the system.**